In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# setting Jedha color palette as default
pio.templates["jedha"] = go.layout.Template(
    layout_colorway=["#4B9AC7", "#4BE8E0", "#9DD4F3", "#97FBF6", "#2A7FAF", "#23B1AB", "#0E3449", "#015955"]
)
pio.templates.default = "jedha"

In [2]:
df = pd.read_csv('Speed+Dating+Data.csv', encoding='latin1')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8378 entries, 0 to 8377
Columns: 195 entries, iid to amb5_3
dtypes: float64(174), int64(13), object(8)
memory usage: 12.5+ MB


In [4]:
df.describe()

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
count,8378.000000,8377.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,6532.000000,8378.000000,...,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000
mean,283.675937,8.960248,0.500597,17.327166,1.828837,11.350919,16.872046,9.042731,9.295775,8.927668,...,7.240312,8.093357,8.388777,7.658782,7.391545,6.810020,7.615079,7.932540,7.155258,7.048611
std,158.583367,5.491329,0.500029,10.940735,0.376673,5.995903,4.358458,5.514939,5.650199,5.477009,...,1.576596,1.610309,1.459094,1.744670,1.961417,1.507341,1.504551,1.340868,1.672787,1.717988
min,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,5.000000,1.000000,1.000000,1.000000,...,2.000000,2.000000,3.000000,2.000000,1.000000,2.000000,2.000000,4.000000,1.000000,1.000000
25%,154.000000,4.000000,0.000000,8.000000,2.000000,7.000000,14.000000,4.000000,4.000000,4.000000,...,7.000000,7.000000,8.000000,7.000000,6.000000,6.000000,7.000000,7.000000,6.000000,6.000000
50%,281.000000,8.000000,1.000000,16.000000,2.000000,11.000000,18.000000,8.000000,9.000000,8.000000,...,7.000000,8.000000,8.000000,8.000000,8.000000,7.000000,8.000000,8.000000,7.000000,7.000000
75%,407.000000,13.000000,1.000000,26.000000,2.000000,15.000000,20.000000,13.000000,14.000000,13.000000,...,8.000000,9.000000,9.000000,9.000000,9.000000,8.000000,9.000000,9.000000,8.000000,8.000000
max,552.000000,22.000000,1.000000,44.000000,2.000000,21.000000,22.000000,22.000000,22.000000,22.000000,...,12.000000,12.000000,12.000000,12.000000,12.000000,10.000000,10.000000,10.000000,10.000000,10.000000


In [5]:
df.columns.tolist()

['iid',
 'id',
 'gender',
 'idg',
 'condtn',
 'wave',
 'round',
 'position',
 'positin1',
 'order',
 'partner',
 'pid',
 'match',
 'int_corr',
 'samerace',
 'age_o',
 'race_o',
 'pf_o_att',
 'pf_o_sin',
 'pf_o_int',
 'pf_o_fun',
 'pf_o_amb',
 'pf_o_sha',
 'dec_o',
 'attr_o',
 'sinc_o',
 'intel_o',
 'fun_o',
 'amb_o',
 'shar_o',
 'like_o',
 'prob_o',
 'met_o',
 'age',
 'field',
 'field_cd',
 'undergra',
 'mn_sat',
 'tuition',
 'race',
 'imprace',
 'imprelig',
 'from',
 'zipcode',
 'income',
 'goal',
 'date',
 'go_out',
 'career',
 'career_c',
 'sports',
 'tvsports',
 'exercise',
 'dining',
 'museums',
 'art',
 'hiking',
 'gaming',
 'clubbing',
 'reading',
 'tv',
 'theater',
 'movies',
 'concerts',
 'music',
 'shopping',
 'yoga',
 'exphappy',
 'expnum',
 'attr1_1',
 'sinc1_1',
 'intel1_1',
 'fun1_1',
 'amb1_1',
 'shar1_1',
 'attr4_1',
 'sinc4_1',
 'intel4_1',
 'fun4_1',
 'amb4_1',
 'shar4_1',
 'attr2_1',
 'sinc2_1',
 'intel2_1',
 'fun2_1',
 'amb2_1',
 'shar2_1',
 'attr3_1',
 'sinc3_1',
 

In [6]:
# L'iid 118 est manquant, filtre les lignes où l'id du partenaire est NaN mais devrait être 118.
df_processed = df[df["pid"].notna()].copy() 

# Rendre le format de "income" utilisable
df_processed["income"] = df_processed["income"].astype(str).apply(lambda x : x.replace(",","")).astype(float)

# Caster certaines colonnes en str afin d'éviter d'appliquer des traitements numériques sur des données qualitatives.
categorical_column = ["iid", "id", "idg","wave", "position", "positin1", "partner", "pid", "race_o", "race", "length", "numdat_2"]
def categorical_to_str(column):
    if column.name in categorical_column :
        return column.astype(str)
    else :
        return column
df_processed = df_processed.apply(categorical_to_str)

fields = {
    1: "Droit",
    2: "Mathématiques",
    3: "Sciences sociales / Psychologie",
    4: "Sciences médicales, pharmaceutiques et biotechnologie",
    5: "Ingénierie",
    6: "Anglais / Écriture créative / Journalisme",
    7: "Histoire / Religion / Philosophie",
    8: "Commerce / Économie / Finance",
    9: "Éducation / Académie",
    10: "Sciences biologiques / Chimie / Physique",
    11: "Travail social",
    12: "Premier cycle / Indécis",
    13: "Sciences politiques / Affaires internationales",
    14: "Cinéma",
    15: "Beaux-arts / Administration des arts",
    16: "Langues",
    17: "Architecture",
    18: "Autre"
}
goals = {
    1: "Ça semblait être une soirée amusante",
    2: "Pour rencontrer de nouvelles personnes",
    3: "Pour avoir un rendez-vous",
    4: "À la recherche d'une relation sérieuse",
    5: "Pour dire que je l'ai fait",
    6: "Autre"
}
frequencies = {
    1: "Plusieurs fois par semaine",
    2: "Deux fois par semaine",
    3: "Une fois par semaine",
    4: "Deux fois par mois",
    5: "Une fois par mois",
    6: "Plusieurs fois par an",
    7: "Presque jamais"
}
career = {
    1: "Avocat",
    2: "Universitaire / Recherche",
    3: "Psychologue",
    4: "Médecin",
    5: "Ingénieur",
    6: "Arts créatifs / Divertissement",
    7: "Banque/Conseil/Finance/Marketing/Affaires/PDG/Entrepreneur/Administration",
    8: "Immobilier",
    9: "Affaires internationales / humanitaires",
    10: "Indécis",
    11: "Travail social",
    12: "Orthophonie",
    13: "Politique",
    14: "Sports professionnels / Athlétisme",
    15: "Autre",
    16: "Journalisme",
    17: "Architecture"
}
df_processed["field_cd"] = df_processed["field_cd"].apply(lambda x : x if pd.isna(x) else fields[x])
df_processed["goal"] = df_processed["goal"].apply(lambda x : x if pd.isna(x) else goals[x])
df_processed["date"] = df_processed["date"].apply(lambda x : x if pd.isna(x) else frequencies[x])
df_processed["go_out"] = df_processed["go_out"].apply(lambda x : x if pd.isna(x) else frequencies[x])
df_processed["career_c"] = df_processed["career_c"].apply(lambda x : x if pd.isna(x) else career[x])


# age_o : age of partner
df_processed["age_dif"] = df_processed["age_o"] - df_processed["age"]

# income : Median household income
df_processed["income_o"] = df_processed["pid"].astype(float).apply(lambda x : df_processed[df_processed["iid"].astype(float) == x]["income"].iloc[0])
df_processed["income_dif"] = df_processed["income_o"] - df_processed["income"]

# Remplacer num_in_3 par numdat_3 dans les lignes présentant des incohérences. Le nombre de dates devrait toujours être supérieur ou égal au nombre matches avec lesquels l'utilisateur a eu au moins un rendez-vous
df_processed.loc[df_processed['num_in_3'] < df_processed['numdat_3'], 'num_in_3'] = df_processed.loc[df_processed['num_in_3'] < df_processed['numdat_3'], 'numdat_3'] 

# Remplacer date_3 par 1 quand numdate_3 est superieur à 0. Si le nombre de dates est supérieur à 0, cela indique qu'il y a eu au moins un rendez-vous
df_processed.loc[df_processed['numdat_3'] > 0, ['date_3']] = 1

# Remplacez les 1 dans date_3 par des 0 lorsque numdat_3 est égal à 0. Si le nombre de dates est égal à 0, cela signifie qu'il n'y a pas eu de rendez-vous. Les zéros dans date_3 sont considérés comme une réponse négative, car il n'y a aucun 2.
df_processed.loc[(df_processed['date_3'] == 1) & (df_processed['numdat_3'] == 0), 'date_3'] = 0

# Remplacez les Nan dans numdat_3 et num_in_3 par des 0 lorsque date_3 est égal à 0. Les NaN sont considérés comme une absence de réponse due au fait que date_3 a reçu une réponse négative.
df_processed.loc[(df_processed['date_3'] == 0), ['numdat_3', "num_in_3"]] = df_processed.loc[(df_processed['date_3'] == 0), ['numdat_3', "num_in_3"]].fillna(0)

# Supprimer les lignes où wave est compris entre 6 et 9. La "préference scale" est décrite comme une échelle de 1 à 10, mais les valeurs sont en réalité comprises entre 0 et 100
df_100p_wave = df_processed[~df_processed['wave'].astype(int).between(6, 9)]

# regrouper par iid, pour éviter les biais liés au nombre variable de participants par vague."
df_Grouped_By_iid = df_processed.groupby("iid").agg(lambda x: x.mean() if pd.api.types.is_numeric_dtype(x) else x.iloc[0]).reset_index()
df_Grouped_By_iid_sum = df_processed.groupby("iid").agg(lambda x: x.sum() if pd.api.types.is_numeric_dtype(x) else x.iloc[0]).reset_index()

df_Grouped_By_iid_100p_wave = df_100p_wave.groupby("iid").agg(lambda x: x.mean() if pd.api.types.is_numeric_dtype(x) else x.iloc[0]).reset_index()

In [7]:
missing_values = df_processed.isnull().sum()

fig = px.bar(missing_values[missing_values > len(df_processed)*0.5], 
             title="Colonnes présentant plus de 50% de valeurs manquantes")
fig.update_layout(showlegend=False)
fig.show()

In [8]:
# Utilisation du DataFrame groupé, car les notes sont les mêmes quel que soit le partenaire.
# Utilisation du DataFrame 100p, car "1_1" et "2_1" sont notés sur 100.

df_Grouped_M = df_Grouped_By_iid_100p_wave[df_Grouped_By_iid_100p_wave["gender"] == 1] 
df_Grouped_F = df_Grouped_By_iid_100p_wave[df_Grouped_By_iid_100p_wave["gender"] == 0]

# 1_1 : We want to know what you look for in the opposite sex.
# Attractive, Sincere, Intelligent, Fun, Ambitious, Shared, Interests
attributes = ["attr","sinc","intel","fun","amb","shar"]
attribute_names = ["Attractivité", "Sincérité", "Intelligence", "divertissant", "Ambition", "Intérêts partagés"]
y_values_M = []
y_values_F = []
for col_name in attributes :
    y_values_M.append(df_Grouped_M[f"{col_name}1_1"].median()) # Utilisation de la médiane pour éviter les biais induits par les outliers.
    y_values_F.append(df_Grouped_F[f"{col_name}1_1"].median()) 



fig = go.Figure()

fig.add_trace(go.Bar(
    x=attribute_names,
    y=y_values_M,
    text=[str(round(num)) for num in y_values_M],
    textposition='auto',
    name='Homme'
))

fig.add_trace(go.Bar(
    x=attribute_names,
    y=y_values_F,
    text=[str(round(num)) for num in y_values_F],
    textposition='auto',
    name='Femme'
))

fig.update_layout(
    title='Comparaison des préférences des femmes et des hommes avant l’événement',
    xaxis_title='Attributs',
    yaxis_title='Note médiane',
    barmode='group'
)

fig.show()

Les attributs les moins recherchés par les hommes sont l'ambition et les intérêts partagés, et il en va de même chez les femmes. 

L'attractivité semble être la priorité absolue pour les hommes, tandis que l'intelligence et la sincérité constituent les qualités les plus recherchées chez les femmes.

In [9]:
# 2_1 : What do you think the opposite sex looks for in a date?

y_values_M_2 = []
y_values_F_2 = []
for col_name in attributes :
    y_values_M_2.append(df_Grouped_M[f"{col_name}2_1"].median()) 
    y_values_F_2.append(df_Grouped_F[f"{col_name}2_1"].median()) 


fig = go.Figure()


fig.add_trace(go.Bar(
    x=attribute_names,
    y=y_values_F_2,
    text=[str(round(num)) for num in y_values_F_2],
    textposition='auto',
    name='Préférence masculine supposée par les femmes'
))

fig.add_trace(go.Bar(
    x=attribute_names,
    y=y_values_M,
    text=[str(round(num)) for num in y_values_M],
    textposition='auto',
    name='Préférence masculine'
))

fig.add_trace(go.Bar(
    x=attribute_names,
    y=y_values_M_2,
    text=[str(round(num)) for num in y_values_M_2],
    textposition='auto',
    name='Préférence féminine supposée par les hommes'
))
fig.add_trace(go.Bar(
    x=attribute_names,
    y=y_values_F,
    text=[str(round(num)) for num in y_values_F],
    textposition='auto',
    name='Préférence féminine'
))


fig.update_layout(
    title="Comparaison des préférences supposées du sexe opposé",
    xaxis_title="Attributs",
    yaxis_title="Note médiane",
    barmode="group"
)

fig.show()

Lorsqu'on compare les préférences des hommes et des femmes avec la manière dont ils pensent que le sexe opposé valorise ces traits, il apparaît clairement que les deux genres ont tendance à surestimer l'importance de l'attractivité. À l'inverse, ils sous-estiment l'importance de la sincérité et de l'intelligence.

In [10]:
# dec : Circle “Yes” or “No” below the ID number of each person you meet to indicate whetheror not you would like to see him or her again, 1=yes 0=no

df_processed_F = df_processed[df_processed["gender"] == 0].copy()
df_processed_M = df_processed[df_processed["gender"] == 1].copy()


cols = attributes + ['like']
cols_names = attribute_names + ["Note globale"]

# round pour obtenir 10 points sur l'axe x.
df_processed_F[cols] = df_processed_F[cols].round(0)
df_processed_M[cols] = df_processed_M[cols].round(0)

fig = go.Figure()

for i in range(len(cols)) :
    positive_dec_percentage_per_rating = df_processed_F.groupby(cols[i])['dec'].mean() * 100
    fig.add_trace(go.Scatter(
        x=positive_dec_percentage_per_rating.index,
        y=positive_dec_percentage_per_rating.values,
        mode='lines+markers',
        name=cols_names[i]
    ))

fig.update_layout(
    title="Pourcentage de décisions positives en fonction des notes données (femmes)",
    xaxis_title="Note",
    yaxis_title="Pourcentage de décisions positives (%)",
    template="plotly"
)

fig.show()


fig = go.Figure()

for i in range(len(cols)) :
    positive_dec_percentage_per_rating = df_processed_M.groupby(cols[i])['dec'].mean() * 100
    fig.add_trace(go.Scatter(
        x=positive_dec_percentage_per_rating.index,
        y=positive_dec_percentage_per_rating.values,
        mode='lines+markers',
        name=cols_names[i]
    ))

fig.update_layout(
    title="Pourcentage de décisions positives en fonction des notes données (hommes)",
    xaxis_title="Note",
    yaxis_title="Pourcentage de décisions positives (%)",
    template="plotly"
)

fig.show()

Toutes les notes semblent corrélées positivement avec la décision, bien que l'attractivité, les intérêts partagés et le divertissement semblent plus important.

Les résultats pour les femmes et les hommes ne présentent pas de différences significatives.

In [28]:
# age_dif : age of partner - age
# income : Median household income based on zipcode using the Census Bureau website
# income_dif : income of partner - income
# int_corr : correlation between participant’s and partner’s ratings of interests inTime 1
# goal : What is your primary goal in participating in this event?
# date : In general, how frequently do you go on dates ?
# go_out : How often do you go out (not necessarily on dates)?
# field_cd : field of study coded
# career_c : What is your intended career? coded

cols = ['age', 'age_dif', 'income', 'income_dif', 'int_corr', 'goal', 'date', 'go_out', 'field_cd', 'career_c']
cols_names = [
    "Âge",
    "Différence d'âge",
    "Revenu médian des ménages",
    "Écart de revenu médian",
    "Corrélation des centres d'intérêt",
    "Raison de participation",
    "Fréquence de date",
    "Fréquence de sortie",
    "Domaine d'études",
    "Carrière"
]

cols_titles = [
    "de l'âge", 
    "de la différence d'âge", 
    "du revenu médian des ménages", 
    "de l'écart de revenu médian", 
    "de la corrélation entre les centres d'intérêt",
    "de la raison de participation", 
    "de la fréquence de date", 
    "de la fréquence de sortie", 
    "du domaine d'études",
    "de la carrière"
]


fig = go.Figure()
buttons = []
for i in range(len(cols)) :
    col_name = cols[i]
    visible = i == 0
    is_numeric = pd.api.types.is_numeric_dtype(df_processed[col_name])
    if is_numeric:
        col_min = df_processed[col_name].min() -0.3
        col_max = df_processed[col_name].max() +0.3
        step_10 = (col_max - col_min)/9
             
        bins = np.arange(col_min, col_max + step_10, step_10)
        group = df_processed.groupby(pd.cut(df_processed[col_name], bins=bins,include_lowest=True), observed=True)
        if col_max > 1000 :
            x_labels = [f"{int(np.round(interval.left / 1000))} k - {int(np.round(interval.right / 1000))} k"for interval in group.groups.keys()]
        else : 
            x_labels = [f"{int(np.round(interval.left))} - {int(np.round(interval.right))}"for interval in group.groups.keys()]
    else : 
        group = df_processed.groupby(col_name)
        x_labels = group.groups.keys()
    group_counts = group.size()
    positive_dec_o_percentage = group['dec_o'].mean() * 100
    positive_dec_percentage = group["dec"].mean() * 100
    match_percentage = group['match'].mean() * 100
    if not is_numeric:
        # Sort based on positive_dec_o_percentage quand la col n'est pas numérique (descending order)
        sorted_idx = positive_dec_o_percentage.sort_values(ascending=False).index
        positive_dec_o_percentage = positive_dec_o_percentage.loc[sorted_idx]
        positive_dec_percentage = positive_dec_percentage.loc[sorted_idx]  
        match_percentage = match_percentage.loc[sorted_idx]

        x_labels = match_percentage.index.astype(str)
        x_labels = [f"{label}{'*' if group_counts[label] < 100 else ''}" for label in x_labels]
    # Obtain the count of rows for each group.
    else :
        x_labels = [f"{label}{'*' if count < 100 else ''}" for label, count in zip(x_labels, group_counts.values)]
    fig.add_trace(go.Scatter(
        x=x_labels,
        y=positive_dec_o_percentage.values,
        mode='lines+markers',
        name="décision positive du partenaire",
        visible=visible
    ))
    fig.add_trace(go.Scatter(
        x=x_labels,
        y=positive_dec_percentage.values,
        mode='lines+markers',
        name="décision positive",
        visible=visible
    ))
    fig.add_trace(go.Scatter(
        x=x_labels,
        y=match_percentage.values,
        mode='lines+markers',
        name="match",
        visible=visible
    ))
    buttons.append({
        "args": [
            {"visible": [True if j in range(i * 3, i * 3 + 3) else False for j in range(len(cols) * 3)] + [True]},
            {"title": f"Pourcentage de décisions positives / match en fonction {cols_titles[i]}",
             "xaxis": {"title": {"text": cols_names[i]}}}
        ],
        "label": cols_names[i],
        "method": "update"
    })


# ajoute une description à *
fig.add_trace(go.Scatter(
    x=[None],
    y=[None],
    mode='markers',
    marker=dict(color='rgba(0,0,0,0)'),
    showlegend=True,
    name="*Groupe de taille réduite (< 100 lignes)."
))



fig.update_layout(
    height=900,
    title=f"Pourcentage de décisions positives / match  en fonction {cols_titles[0]}",
    xaxis_title="Age",
    yaxis_title="Pourcentage de décisions positives / match (%)",
    margin=dict(b=275),
    updatemenus=[
        dict(
            type="buttons",
            direction="down",
            buttons=buttons,
        )
    ],
    legend=dict(
        xanchor="right", 
        yanchor="bottom" 
    )
)

fig.show()

Les participants plus âgés obtiennent légèrement moins de décisions positives. Le graphique sur la différence d'âge montre que les chances de match augmentent lorsque cette différence est proche de zéro. Les participants donnent davantage de décisions favorables lorsque leur partenaire est plus jeune

Le revenu médian des ménages semble avoir peu d'impact sur les décisions, bien qu'une légère corrélation semble apparaître entre une décision positive du partenaire et un revenu plus élevé de celui-ci par rapport au participant.

La corrélation entre les centres d'intérêt exerce un léger effet positif sur la décision.

Les individus présents sans objectif précis reçoivent légèrement plus de décisions positives de la part de leurs partenaires. 
Ceux qui sont habitués aux rendez-vous et aux sorties se révèlent nettement plus performants et davantage enclins à valider leurs partenaires, ce qui conduit à un nombre de matchs significativement plus élevé.

In [24]:
# expnum: Out of the 20 people you will meet, how many do you expect will be interested in dating you ? signup/Time1
# match_es : How many matches do you estimate you will get (a match occurs when you and yourpartner both check “Yes” next to decision) ?  just after speed date
# match 1=yes, 0=no

avg_dif_match_and_expected_before = (df_Grouped_By_iid["expnum"] - df_Grouped_By_iid["match"]).median()
avg_dif_match_and_expected_after = (df_Grouped_By_iid["match_es"] - df_Grouped_By_iid["match"]).median()



# 3_ How do you think you measure up ? 
# 5_ And finally, how do you think others perceive you ? 
# _o rating by partner the night of the event, for all 6 attributes
# _1 signup/Time 1 : Survey filled out by students that are interested in participating in order to register forthe event.
# _s Half way through meeting all potential dates during the night of the event on theirscorecard
# _2 followup/Time2 : Survey is filled out the day after participating in the event. Subjects must have submitted this in order to be sent their matches.
# _3 followup2/ Time3 : Subjects filled out 3-4 weeks after they had been sent their matches

y_values_3_1 = []
y_values_3_s = []
y_values_3_2 = []
y_values_3_3 = []

y_values_5_1 = []
y_values_5_2 = []
y_values_5_3 = []

for col_name in attributes[:-1] :
    y_values_3_1.append((df_Grouped_By_iid_100p_wave[f"{col_name}3_1"] - df_Grouped_By_iid_100p_wave[f"{col_name}_o"]).mean())
    y_values_3_s.append((df_Grouped_By_iid_100p_wave[f"{col_name}3_s"] - df_Grouped_By_iid_100p_wave[f"{col_name}_o"]).mean())
    y_values_3_2.append((df_Grouped_By_iid_100p_wave[f"{col_name}3_2"] - df_Grouped_By_iid_100p_wave[f"{col_name}_o"]).mean())
    y_values_3_3.append((df_Grouped_By_iid_100p_wave[f"{col_name}3_3"] - df_Grouped_By_iid_100p_wave[f"{col_name}_o"]).mean())
    
    y_values_5_1.append((df_Grouped_By_iid_100p_wave[f"{col_name}5_1"] - df_Grouped_By_iid_100p_wave[f"{col_name}_o"]).mean())
    y_values_5_2.append((df_Grouped_By_iid_100p_wave[f"{col_name}5_2"] - df_Grouped_By_iid_100p_wave[f"{col_name}_o"]).mean())
    y_values_5_3.append((df_Grouped_By_iid_100p_wave[f"{col_name}5_3"] - df_Grouped_By_iid_100p_wave[f"{col_name}_o"]).mean())


fig = go.Figure()
fig.add_trace(go.Bar(
    x=["Nombre de matchs attendus avant l'événement moins le nombre de matchs obtenus", "Nombre de matchs attendus après l'événement moins le nombre de matchs obtenus"],
    y=[avg_dif_match_and_expected_before, avg_dif_match_and_expected_after],
    text=[f'{avg_dif_match_and_expected_before:.1f}', f'{avg_dif_match_and_expected_after:.1f}'],
    name="",
    textposition='auto'
))

fig.add_trace(go.Bar(
    x=attribute_names[:-1],
    y=y_values_3_1,
    text=[str(round(num,1)) for num in y_values_3_1],
    textposition='auto',
    name="Avant l'évenement",
    visible= False
))
fig.add_trace(go.Bar(
    x=attribute_names[:-1],
    y=y_values_3_s,
    text=[str(round(num,1)) for num in y_values_3_s],
    textposition='auto',
    name="Pendant l'évenement",
    visible= False
))
fig.add_trace(go.Bar(
    x=attribute_names[:-1],
    y=y_values_3_2,
    text=[str(round(num,1)) for num in y_values_3_2],
    textposition='auto',
    name="Un jour après l'évenement",
    visible= False
))
fig.add_trace(go.Bar(
    x=attribute_names[:-1],
    y=y_values_3_3,
    text=[str(round(num,1)) for num in y_values_3_3],
    textposition='auto',
    name="Plusieurs semaines après l'évenement",
    visible= False
))

fig.add_trace(go.Bar(
    x=attribute_names[:-1],
    y=y_values_5_1,
    text=[str(round(num,1)) for num in y_values_5_1],
    textposition='auto',
    name="Avant l'évenement",
    visible= False
))
fig.add_trace(go.Bar(
    x=attribute_names[:-1],
    y=y_values_5_2,
    text=[str(round(num,1)) for num in y_values_5_2],
    textposition='auto',
    name="Un jour après l'évenement",
    visible= False
))
fig.add_trace(go.Bar(
    x=attribute_names[:-1],
    y=y_values_5_3,
    text=[str(round(num,1)) for num in y_values_5_3],
    textposition='auto',
    name="Plusieurs semaines après l'évenement",
    visible= False
))


fig.update_layout(
    showlegend=False,
    title='Matchs prévus vs matchs réellement obtenus',
    yaxis_title='Différence moyenne',
    updatemenus=[
        dict(
            type="buttons",
            direction="down",
            buttons=[{
                "args": [
                    {"visible": [True] + [False for i in range(7)]},
                    {"title": "Matchs prévus vs matchs réellement obtenus",
                    "xaxis": {"title": {"text": "Match"}},
                    "showlegend": False}
                ],
                "label": "Match",
                "method": "update"
            },{
                "args": [
                    {"visible": [False] + [True if i < 4 else False for i in range(7)]},
                    {"title": "Moyenne de la différence entre l'auto-évaluation et la moyenne des évaluations des autres",
                    "xaxis": {"title": {"text": "Attribut"}},
                    "showlegend": True}
                ],
                "label": "Auto-évaluation ",
                "method": "update"
            },{
                "args": [
                    {"visible": [False] + [True if i > 3 else False for i in range(7)]},
                    {"title": "Moyenne de la différence entre la prédiction des évaluations externes et la moyenne des évaluations des autres.",
                    "xaxis": {"title": {"text": "Attribut"}},
                    "showlegend": True}
                ],
                "label": "Prédiction des évaluations externes",
                "method": "update"
            }
            
            
            ],
        )
    ],
    legend=dict(
        xanchor="right",
        yanchor="bottom" 
    )

)


fig.show()

En comparant les auto-évaluations avec les évaluations des partenaires, il apparaît que les individus ont une tendance à surestimer leur propre valeur perçue. 
En moyenne, les participants surestiment de 3,9 le nombre de matchs qu'ils pensent obtenir, une surestimation qui diminue à 2,4 après le speed dating. Ils évaluent leurs propres caractéristiques personnelles à environ 1 point de plus que la moyenne des notes attribuées par leurs partenaires. Cette surestimation s'atténue après l'événement et continue de décroître au fil des semaines.

In [70]:
# order: the number of date that night when met partner
# Waves have varying numbers of participants, resulting in different maximum "order".


# Filter first and last dates
first_last = df_processed[(df_processed['order'] == 1) | (df_processed['order'] == df_processed.groupby('wave')['order'].transform('max'))].copy()

# Add a column indicating if it's a first or last date
first_last['type'] = first_last['order'].apply(lambda x: '1er date' if x == 1 else 'Dernier date')


# Filter the intermediate dates (neither first nor last)
others = df_processed[
    (df_processed['order'] != 1) & 
    (df_processed['order'] != df_processed.groupby('wave')['order'].transform('max'))
].copy()

others['type'] = "Date intermédiaire"

# Combine both datasets
all_dates = pd.concat([first_last, others], ignore_index=True)


# Analyze the decision (dec == 1 means yes for a second date)
grouped_decision = all_dates.groupby('type').agg({'dec': 'mean'}).reset_index() 

# Analyze the match rate
grouped_match = all_dates.groupby('type').agg({'match': 'mean'}).reset_index() 

# Create subplots (1 row, 2 columns)
fig = make_subplots(rows=1, cols=2, subplot_titles=('Décision positive', 'Match'))

# Add the first subplot (second date decision rate)
fig.add_trace(go.Bar(
    x=grouped_decision['type'],
    y=grouped_decision['dec'] * 100,
    text=(grouped_decision['dec'] * 100).round(1),
    name='Decision Rate',
), row=1, col=1)

# Add the second subplot (match rate)
fig.add_trace(go.Bar(
    x=grouped_match['type'],
    y=grouped_match['match'] * 100,
    text= (grouped_match['match'] * 100).round(1),
    name='Match Rate',
), row=1, col=2)
# Update layout
fig.update_layout(
    title='Comparaison du taux de match et décision positive en fonction de la position lors du speed dating.',
    showlegend=False,  # Hide the legend as the names are similar
)

# Add y-axis labels
fig.update_yaxes(title_text="Taux (%)", row=1, col=1)
fig.update_yaxes(title_text="Taux (%)", row=1, col=2)


fig.show()


Pour obtenir un deuxième rendez-vous, il est légèrement avantageux d'être le premier speed date de la soirée plutôt que le dernier. Quoi qu'il en soit, tant le premier que le dernier speed date affichent globalement de meilleures chances de succès que ceux occupant des positions intermédiaires.